In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"
# set jax number of CPU devices
# ncpus = 32
# os.environ['XLA_FLAGS'] = f"--xla_force_host_platform_device_count={ncpus}"

# from functools import partial
# import numpy as np

# import numba

import jax
import jax.numpy as jnp
# from jax.sharding import Mesh, PartitionSpec as P
# from jax.experimental.shard_map import shard_map

# numba.set_num_threads(ncpus)

jax.config.update("jax_enable_x64", True)
# jax.config.update("jax_platforms", 'cpu')

In [ ]:
# @numba.guvectorize([(numba.float64, numba.float64, numba.float64[:])], '(),()->()',nopython=True,target='parallel')
# def add_two_floats(a, b, res):
#     res[0] = a + b

# N = 10_000_000
# inputs = np.random.random(N)

# outputs = add_two_floats(inputs, inputs)
# print(outputs[0], outputs[-1])

In [ ]:
# # 1. Setup the Mesh
# num_devices = jax.local_device_count()
# devices = jax.devices()
# mesh = jax.make_mesh((num_devices,), ('data_parallel',))

# # 2. Define the Kernel Logic
# def add_logic(a_shard, b_shard):
#     return a_shard + b_shard

# # 3. Create the Sharded Function using shard_map
# # We pass the mesh and specs directly to the function
# parallel_add = shard_map(
#     add_logic,
#     mesh=mesh,
#     in_specs=(P('data_parallel'), P('data_parallel')),
#     out_specs=P('data_parallel')
# )

# result = parallel_add(inputs, inputs)

In [3]:
# 1. Create a PRNG key (seed)
key = jax.random.PRNGKey(42)

# 2. Use jax.random to generate data
N = 5_000_000
# We use the key to generate the specific array
inputs = jax.random.normal(key, shape=(N, 3))

# Your existing batch-processed operation
def library_kernel(v):
    return jnp.exp(-jnp.linalg.norm(v)**2) * v

@jax.jit
def batch_processed_op(vectors):
    return jax.lax.map(library_kernel, vectors, batch_size=1024)

results = batch_processed_op(inputs)

In [4]:
results

Array([[-0.18893992, -0.08954847, -0.01597509],
       [ 0.00452096,  0.00197375, -0.00168866],
       [ 0.42229236, -0.06787512, -0.02860335],
       ...,
       [ 0.0136736 ,  0.03481165, -0.02586038],
       [-0.30706099,  0.1183369 , -0.26277664],
       [-0.09176462, -0.27103137,  0.23886354]], dtype=float64)